In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(survminer)
library(rlang)
library(cowplot)
library(patchwork)
library(ggpubr)

# 0 - Read in base data

In [ ]:
univariate_ready <- 
fread(paste0(SHARE_DIR, "fisher_base.csv")) %>% 
 rename_with(~ gsub("_geneset", "", .x)) %>% 
 rename_with(~ gsub("_AND_HYPOTAURINE", "", .x)) %>% 
 mu(cohortGo = gsub(" ## ", " / ", cohortGo))

In [ ]:
combos_ready <- readRDS(paste0(TMP_DIR, "combo_features.Rds"))

In [ ]:
go <- 
fread(paste0(SHARE_DIR,"share_highlights_with_fran_update.csv")) %>%
 mu(feature = gsub("_AND_HYPOTAURINE", "", feature)) %>% 
 tm( cohortGo = gsub(" ## ", " / ", cohortGo), feature = gsub("_geneset", "", feature), type) 

In [ ]:
univariate <- go %>% fi(type == "univariate")
combo <- go %>% fi(type == "combination")

# 1 - Survival Images

In [ ]:
data_prep <- function(x, cohort, type = "univariate") {
    if(type == "univariate"){
        data <- ready %>% fi( cohortGo == !! cohort) %>% drop_na( !!sym(x) )
    } else {
        data <- combos_ready[[ gsub(" / ", " ## ", cohort)]] %>% rename_with(~ gsub("_geneset", "", .x)) %>% drop_na( !!sym(x) )
    }
    surv_formula <- expr(Surv(daysToPfsEvent, pfsEvent) ~ !!sym(x))
    fits <- survfit(eval(surv_formula, envir = environment()), data = data)
    pval <- signif(survdiff(eval(surv_formula, envir = environment()), data = data)$pvalue,2)

    list("data" = data, "fits" = fits, "surv_formula" = surv_formula, "pval" = pval)
}

In [ ]:
figurer <- function(x, cohort, data, fits, pval, pad = 30 ) {
        
    # Compute max time in data
    max_time <- max(data$daysToPfsEvent, na.rm = TRUE)
    xmax <- min(max_time, 1100)
    
    #fit
    oo <- 
    ggsurvplot(
     fits,
     data = data,
     palette = c("#7AABD3", "#e52f28"),
     conf.int = TRUE, 
     risk.table = TRUE, 
     pval.coord = c(700, .95), 
     xlim = c(0, xmax), 
     break.time.by = 300, 
     ggtheme = theme_minimal(),
     xlab = "Days", 
     ylab = "Survival Probability", 
     title = cohort) 

    oo$plot <- 
    oo$plot + 
     annotate("text", x = 500, y = 0.9, label = paste0("Log-rank        \np-value = ", pval), size = 5) + 
     theme(plot.title = element_text(hjust = 0.5)) + 
     guides(color = guide_legend(nrow = 2, byrow = TRUE)) 

    as_ggplot(gridExtra::arrangeGrob(oo$plot, oo$table, layout_matrix = matrix(c(1,1,1,1,1,1,1,2,2)))) + 
    theme(plot.margin = margin(t = pad, r = pad, b = pad, l = pad))
}

In [ ]:
survival_figures <- list()
for( i in seq(nrow(go))){
    cohortGo <- as.character(go[i,1])
    feature <- as.character(go[i,2])
    type <- as.character(go[i,3])

    data_ready <- data_prep( feature, cohortGo, type)

    data <- data_ready$data
    fits <- data_ready$fits
    pval <- data_ready$pval
    surv_formula <- data_ready$surv_formula

    survival_figures[[i]] <- figurer2( feature, cohortGo, data = data, fits = fits, pval = pval)
}

In [ ]:
options(repr.plot.height = 26, repr.plot.width = 26) 
ggsave(paste0(SHARE_DIR, "5_pfs_1.png"), wrap_plots(survival_figures[1:16], ncol = 4), width = 26, height = 26)
ggsave(paste0(SHARE_DIR, "5_pfs_2.png"), wrap_plots(survival_figures[17:32], ncol = 4), width = 26, height = 26)
ggsave(paste0(SHARE_DIR, "5_pfs_3.png"), wrap_plots(survival_figures[33:48], ncol = 4), width = 26, height = 26)